In [2]:
!pip install avro kafka-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.4/285.4 kB 22.1 MB/s eta 0:00:00


In [3]:
# import required libraries
from kafka import KafkaConsumer, KafkaProducer
import avro.schema
import avro.io
import io
import hashlib, json

In [4]:
def serialize(schema, obj):
    bytes_writer = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_writer)
    writer = avro.io.DatumWriter(schema)
    writer.write(obj, encoder)
    return bytes_writer.getvalue()

In [5]:
def deserialize(schema, raw_bytes):
    bytes_reader = io.BytesIO(raw_bytes)
    decoder = avro.io.BinaryDecoder(bytes_reader)
    reader = avro.io.DatumReader(schema)
    return reader.read(decoder)

In [6]:
schema_file = 'transaction.avsc'
txschema = avro.schema.parse(open(schema_file).read())
schema_file = 'submit.avsc'
submitschema = avro.schema.parse(open(schema_file).read())
schema_file = 'result.avsc'
resultschema = avro.schema.parse(open(schema_file).read())

In [7]:
# Connect to kafka broker running in your local host (docker). Change this to your kafka broker if needed
kafka_broker = 'lab.aimet.tech:9092'

In [8]:
producer = KafkaProducer(bootstrap_servers=[kafka_broker])

In [9]:
txconsumer = KafkaConsumer(
    'transaction',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(txschema, x))
resultconsumer = KafkaConsumer(
    'result',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(resultschema, x))

In [10]:
def gen_signature(txid, payer, payee, amount, token):
    o = {'txid': txid, 'payer': payer, 'payee': payee, 'amount': amount, 'token': token}
    return hashlib.md5(json.dumps(o, sort_keys=True).encode('utf-8')).hexdigest()

In [11]:
# Student credentials
VID = "V632456"
TOKEN = "0cf778b3f1ad9c822ac66ff8c7409dc7"

In [12]:
def verify_transactions():
    print("Starting transaction verification process...")

    # 1. Read from transaction topic
    for transaction_msg in txconsumer:
        transaction = transaction_msg.value
        txid = transaction['txid']
        payer = transaction['payer']
        payee = transaction['payee']
        amount = transaction['amount']

        print(f"Received transaction: {txid}")

        # 2. Generate signature and submit verification
        signature = gen_signature(txid, payer, payee, amount, TOKEN)

        submit_msg = {
            "vid": VID,
            "txid": txid,
            "signature": signature
        }

        # Serialize and send to submit topic
        serialized_submit = serialize(submitschema, submit_msg)
        producer.send('submit', value=serialized_submit)
        print(f"Submitted verification for txid: {txid}")

        # 3. Wait for result
        for result_msg in resultconsumer:
            result = result_msg.value
            if result['vid'] == VID and result['txid'] == txid:
                if result['code'] == 200:
                    print("Verification successful!")
                    print(f"Result details:")
                    print(f"Timestamp: {result['timestamp']}")
                    print(f"VID: {result['vid']}")
                    print(f"TXID: {result['txid']}")
                    print(f"Checksum: {result['checksum']}")

                    # This is what you'll submit to Courseville
                    submission = {
                        "timestamp": result['timestamp'],
                        "vid": result['vid'],
                        "txid": result['txid'],
                        "checksum": result['checksum']
                    }
                    return submission
                else:
                    print(f"Verification failed with code: {result['code']}")
                    print(f"Message: {result['message']}")
                break


In [13]:
result = verify_transactions()
print("\nSubmission for Courseville:")
print(json.dumps(result, indent=2))

Starting transaction verification process...
Received transaction: TX03118
Submitted verification for txid: TX03118
Verification successful!
Result details:
Timestamp: 1744099614
VID: V632456
TXID: TX03118
Checksum: 82264f703e61c1dc7852a11ed2ed46da

Submission for Courseville:
{
  "timestamp": 1744099614,
  "vid": "V632456",
  "txid": "TX03118",
  "checksum": "82264f703e61c1dc7852a11ed2ed46da"
}
